# DSCI 525 - Web and Cloud Computing
## Project: Daily Rainfall Over NSW, Australia
## Milestone 1: Tackling Big Data on Your Laptop 
### Authors: Group 24 Huanhuan Li, Nash Makhija and Nicholas Wu

In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
from memory_profiler import memory_usage
import dask.dataframe as dd

In [2]:
%load_ext rpy2.ipython
%load_ext memory_profiler

## 1) Downloading the data

In [3]:
# Necessary metadata
article_id = 14096681  # this is the unique identifier of the article on figshare
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "../data/"

In [4]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data, feel free to check it out
files = data["files"]             # this is just the data about the files, which is what we want
files

[{'is_link_only': False,
  'name': 'daily_rainfall_2014.png',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'id': 26579150,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'size': 58863},
 {'is_link_only': False,
  'name': 'environment.yml',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'id': 26579171,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'size': 192},
 {'is_link_only': False,
  'name': 'README.md',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'id': 26586554,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'size': 5422},
 {'is_link_only': False,
  'name': 'data.zip',
  'supplied_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'computed_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'id': 26766812,
  'download_url': 'https://

In [6]:
%%time
files_to_dl = ["data.zip"]
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: user 6.26 s, sys: 6.02 s, total: 12.3 s
Wall time: 8min 27s


In [7]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

CPU times: user 17.3 s, sys: 1.93 s, total: 19.3 s
Wall time: 19.5 s


## 2) Combine data CSVs

### 1. Using Pandas

In [8]:
### just listing to get an idea how individual file looks like 
use_cols = ['time', 'lat_min', 'lat_max', 'lon_min', 'lon_max', 'rain (mm/day)']
df = pd.read_csv("../data/ACCESS-CM2_daily_rainfall_NSW.csv", usecols=use_cols)
df

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
0,1889-01-01 12:00:00,-36.25,-35.00,140.625,142.50,3.293256e-13
1,1889-01-02 12:00:00,-36.25,-35.00,140.625,142.50,0.000000e+00
2,1889-01-03 12:00:00,-36.25,-35.00,140.625,142.50,0.000000e+00
3,1889-01-04 12:00:00,-36.25,-35.00,140.625,142.50,0.000000e+00
4,1889-01-05 12:00:00,-36.25,-35.00,140.625,142.50,1.047658e-02
...,...,...,...,...,...,...
1932835,2014-12-27 12:00:00,-30.00,-28.75,151.875,153.75,2.951144e-02
1932836,2014-12-28 12:00:00,-30.00,-28.75,151.875,153.75,2.257118e-01
1932837,2014-12-29 12:00:00,-30.00,-28.75,151.875,153.75,1.204670e-01
1932838,2014-12-30 12:00:00,-30.00,-28.75,151.875,153.75,2.632404e-02


In [20]:
%%time
%memit
# Shows time that regular python takes to merge file
# Join all data together
## here we are using a normal python way of merging the data 
files = glob.glob('../data/*NSW.csv')
df = pd.concat((pd.read_csv(file, index_col=0, usecols=use_cols)
                .assign(model=file[8:file.index("_daily")])
                for file in files)
              )
df.to_csv("../data/combined_data.csv")

peak memory: 4668.88 MiB, increment: 0.25 MiB
CPU times: user 4min 37s, sys: 13 s, total: 4min 50s
Wall time: 4min 55s


In [22]:
%%time

df_pandas = pd.read_csv("../data/combined_data.csv")

CPU times: user 45.4 s, sys: 10.6 s, total: 56.1 s
Wall time: 57.9 s


In [25]:
df_pandas.head()

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
0,1889-01-01 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.244226e-13,MPI-ESM-1-2-HAM
1,1889-01-02 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.217326e-13,MPI-ESM-1-2-HAM
2,1889-01-03 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.498125e-13,MPI-ESM-1-2-HAM
3,1889-01-04 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.251282e-13,MPI-ESM-1-2-HAM
4,1889-01-05 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.270161e-13,MPI-ESM-1-2-HAM


### 2. Using DASK

In [26]:
%%time
%%memit
# shows time that dask take to merge
ddf = dd.read_csv("../data/*NSW.csv",assume_missing=True,usecols=use_cols)
ddf.to_csv("../data/combined_data_dask.csv", single_file=True)

peak memory: 9276.64 MiB, increment: 3411.12 MiB
CPU times: user 6min 18s, sys: 24.3 s, total: 6min 42s
Wall time: 5min 56s


In [27]:
%%time

df_dask = dd.read_csv("../data/combined_data_dask.csv")

CPU times: user 12.7 ms, sys: 5.14 ms, total: 17.9 ms
Wall time: 16 ms


## 3) Load the Combined CSV to Memory and Perform a Simple EDA

## 4) Perform a Simple EDA in R